In [ ]:
import traceback
import socket
from datetime import datetime, timedelta
from typing import Optional, Sequence, List
from rpc.client import RpcClient
from rpc.utility import INTERVAL_ADJUSTMENT_MAP
from rpc.utility import get_duration, extract_vt_symbol, to_rq_symbol, handle_df, load_json


def get_trading_symbols() -> set:
    symbols = set()
    df = all_instruments(date=datetime.now())
    for ix, row in df.iterrows():
        symbols.add(row["order_book_id"])
    return symbols
    
    
def query_by_symbol(vt_symbol: str, rq_interval: str, start_date: datetime, end_date: datetime) -> dict:
    symbol, exchange = extract_vt_symbol(vt_symbol)
    rq_symbol = to_rq_symbol(symbol, exchange)
    adjustment = INTERVAL_ADJUSTMENT_MAP[rq_interval]
    end_date += timedelta(1)

    df = get_price(
        rq_symbol,
        frequency=rq_interval,
        fields=["open", "high", "low", "close", "volume"],
        start_date=start_date,
        end_date=end_date,
        adjust_type="none"
    )
    
    df = handle_df(df, rq_interval)
    return df.to_dict(orient="records")

def init_client(host: str, port: int, authkey: bytes):
    client = RpcClient(host, port, authkey)
    client.connect()
    try:
        return client
    except:
        traceback.print_exc()

        
def get_update_symbol(client: RpcClient) -> List:
    symbols = client.get_update_symbol()
    print("待更新的合约列表获取成功：")
    print(symbols)
    return symbols


def save_all_data(client: RpcClient, rq_interval: str, start: datetime, end: datetime, symbols: Optional[Sequence[str]] = None):
    if symbols is None:
        symbols = get_update_symbol(client)
    for vt_symbol in symbols:
        data_dict = query_by_symbol(vt_symbol, rq_interval, start, end)
        client.save_to_database(data_dict, vt_symbol, rq_interval)
        print(f"{vt_symbol}合约数据保存成功")
        
connect_setting = load_json('connect.json')

In [ ]:
host_home = socket.gethostbyname(connect_setting['host_home'])
host_tencent = connect_setting['host_tencent']
port = connect_setting['port']
authkey = connect_setting['authkey'].encode('ascii')
print(datetime.now(), host_home)

### 分别推送

In [ ]:
def update_to_2server():   
    rq_interval = "60m"
    back_days = 30
    start, end = get_duration(back_days)
    
    client_home = init_client(host_home, port, authkey)
    if client_home:
        save_all_data(client_home, rq_interval, start, end)
        client_home.close()

    client_tencent = init_client(host_tencent, port, authkey)
    if client_tencent:
        save_all_data(client_tencent, rq_interval, start, end)
        client_tencent.close()

    return client_home, client_tencent
    
client_h, client_t = update_to_2server()

### 合并推送

In [ ]:
def main():    
    home_symbols = None
    
    rq_interval = "60m"
    back_days = 30
    start, end = get_duration(back_days)

    
    client_home = init_client(host_home, port, authkey)
    if client_home:
        home_symbols = get_update_symbol(client_home)

    client = init_client(host_tencent, port, authkey)
    if client:
        tencent_symbols = get_update_symbol(client)
        if home_symbols:
            tencent_symbols.extend(home_symbols)
            tencent_symbols = set(tencent_symbols)
            
        print("合并后待更新的合约列表：")
        print(tencent_symbols)
        
        save_all_data(client, rq_interval, start, end, tencent_symbols)
        client.close()

    return client
    
client = main()